<a href="https://colab.research.google.com/github/alejogiley/Hack7/blob/master/Unqualified_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Anaconda

In [0]:
#!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
#!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
#!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

#!conda install -q -y --prefix /usr/local -c omnia --no-update-deps pdbfixer=1.4
#!conda install -q -y --prefix /usr/local -c conda-forge --no-update-deps xgboost=0.6a2
#!conda install -q -y --prefix /usr/local -c rdkit --no-update-deps rdkit=2017.09.1
#!conda install -q -y --prefix /usr/local -c deepchem --no-update-deps  deepchem-gpu=2.1.0

#import sys
#sys.path.append('/usr/local/lib/python3.6/site-packages/')
#ignore warning about PYTHONPATH

In [0]:
import os
import pandas as pd
import numpy as np

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Stats
from scipy.stats import skew

#Importing scikit-learn
import sklearn as sk
#Fill---
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, RobustScaler 
from sklearn.preprocessing import Imputer
#Normalizing
from sklearn import preprocessing
#Data split
from sklearn.model_selection import train_test_split 
#Random Forest packages
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
#Evaluation package
from sklearn.metrics import accuracy_score

#Importing XG_BOOST packages
import xgboost as xg
from xgboost import plot_importance

#Importing Neural Network Packages
from sklearn.neural_network import MLPClassifier

#Importing rdkit packages
from rdkit import RDConfig
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.ML.Descriptors import MoleculeDescriptors

#Importing DeepChem
import deepchem as dc

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
# clean enviroment
lst = [df, pp]
del lst
pp

In [168]:
# Load Dataset
df = pd.read_csv('/content/drive/My Drive/Hack 7/Gyrase/AZ_Pyrrolamides_4OE.csv', encoding='unicode_escape') 
df.head()

,External_ID,Molecule SMILES,First Sample Reg. Date,SAU Gyr IC50 (µM),SAU Gyr Ki (µM),Sau516 MIC (µg/ml),Sau516 MIC (µM),Mean Human Prot binding (% free),Mean LogD,Determination of pKa HA1,Determination of pKa HA2,Determination of pKa B1,Determination of pKa B2,Sirius pKa
0,AZ1001,c1cc(c(nc1)N2CCC(CC2)NC(=O)c3cc(c([nH]3)Cl)Cl)[N+](=O)[O-],6/10/2002,>400,NaN,>64,NaN,NaN,NV,NaN,NaN,NaN,NaN,NaN
1,AZ1002,CC(C)c1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)[N+](=O)[O-])Br,8/16/2002,>50,NaN,>64,NaN,NaN,>4,NaN,NaN,NaN,NaN,NaN
2,AZ1003,CCOC(=O)c1cc(nc(c1C#N)N2CCC(CC2)NC(=O)c3cc(c([nH]3)C)Br)C,8/16/2002,7.14,NaN,>64,NaN,NaN,>3.66,NaN,NaN,NaN,NaN,NaN
3,AZ1004,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)C#N)Br,8/16/2002,9.93,NaN,>64,NaN,NaN,>3.13,NaN,NaN,NaN,NaN,NaN
4,AZ1005,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3ccc4ccccc4n3)Br,8/16/2002,3.31,NaN,>64,NaN,NaN,>4.07,NaN,NaN,NaN,NaN,NaN


In [0]:
# Get Smiles
pp['Smiles'] = df['Molecule SMILES']
pp['ProblematicSmiles'] = pd.Series()

# Flag bad smiles
def smile2mol(x):
  m = Chem.MolFromSmiles(x)
  if m is None:
    return None
  else:
    return m

# Get Mol objects 
pp['Molecule'] = pp['Smiles'].apply(lambda x: smile2mol(x))

# find index of problematic smiles
ps = [int(x) for x in range(len(pp['Mol'])) if pp['Molecule'].loc[x] is None ]

# Fix problematic smiles ??
if len(ps) > 0:
  print("Having problem getting structures for these smiles:")
  pp['ProblematicSmiles'] = pp['Smiles'].loc[ps]
  for i in range(len(ps)):
    print("index: %i" % ps[i], pp['ProblematicSmiles'].loc[ps[i]])

In [170]:
# Descriptors
descriptors = list(np.array(Descriptors._descList)[:,0])
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors) 

# Make is easier to identify problematic molecules later
# e.g infity descriptor values
def computeDescriptors(mol, calculator):
  res = np.array(calculator.CalcDescriptors(mol))
  if not np.all(np.isfinite(res)):
    return None 
  return res 

# Selecting only non-null molecules 
pp = pp[ pp["Molecule"].notnull() ]

# Mapping descriptors with molecules
pp['Descriptors'] = pp['Molecule'].map(lambda x: computeDescriptors(x, calculator)) 

# Flags problems
ps = pp["Descriptors"].isnull()
print("{} molecules failed to get descriptors".format(ps.sum())) 

# Rename columns
des = pp["Descriptors"].apply(pd.Series)
des = des.rename(columns = lambda x : descriptors[x])

0 molecules failed to get descriptors


In [0]:
# Concatenating main and descriptor dataframes
# df['SAU Gyr IC50 (µM)'] = df['SAU Gyr IC50 (µM)'].fillna((df['SAU Gyr IC50 (µM)'].mean()), inplace=True)
dataset = pd.concat([df[["SAU Gyr IC50 (µM)"]], des], axis=1)
dataset = dataset.dropna(how ='any')

In [0]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
for i in to_drop:
    df_final.drop(i, axis=1, inplace=True)

In [0]:
 y = new2['activity_micromolar']
X = new2.drop('activity_micromolar',axis=1)

In [0]:
#data split based on sythesis date into train test and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [0]:
# Define a function to calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

# Define a function to calculate negative RMSE (as a score)
def nrmse(y_true, y_pred):
    return -1.0*rmse(y_true, y_pred)

neg_rmse = make_scorer(nrmse)

estimator = XGR()
selector = RFECV(estimator, cv = 3, n_jobs = -1, scoring = neg_rmse)
selector = selector.fit(X_train, y_train)

print("The number of selected features is: {}".format(selector.n_features_))

features_kept = X_train.columns.values[selector.support_] 

X_train = selector.transform(X_train)  
X_test = selector.transform(X_test)